# Azure ML Connection - Simple Setup

This notebook provides a straightforward way to connect to Azure ML without dealing with CLI issues. We'll use browser-based authentication which is more reliable.

## Step 1: Install Required Packages

First, let's make sure we have the Azure ML SDK installed:

In [1]:
# Install Azure ML SDK packages
!pip install azure-ai-ml azure-identity --quiet

print("✅ Azure ML packages installed!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow-skinny 2.21.3 requires packaging<25, but you have packaging 25.0 which is incompatible.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
jupyter-resource-usage 0.7.2 requires psutil~=5.6, but you have psutil 7.0.0 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
azureml-training-tabular 1.60.0 requires psutil<5.9.4,>=5.2.2, but you have psutil 7.0.0 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-mlflow 1.60.0 requires azure-storage-blob<=12.19.0,>=12.5.0, but you have azure-

## Step 2: Set Your Azure ML Workspace Details

**Replace these values with your actual Azure ML workspace details:**

You can find these in the Azure Portal:
1. Go to [Azure Portal](https://portal.azure.com)
2. Navigate to your Azure Machine Learning workspace
3. Copy the Subscription ID, Resource Group, and Workspace Name from the Overview page

In [ ]:
# 🔧 CONFIGURE THESE VALUES FOR YOUR WORKSPACE
SUBSCRIPTION_ID = "your-subscription-id-here"  # Replace with your subscription ID
RESOURCE_GROUP = "your-resource-group-here"   # Replace with your resource group
WORKSPACE_NAME = "your-workspace-name-here"   # Replace with your workspace name

print(f"Workspace Configuration:")
print(f"  Subscription: {SUBSCRIPTION_ID}")
print(f"  Resource Group: {RESOURCE_GROUP}")
print(f"  Workspace: {WORKSPACE_NAME}")

# Validate that values were changed
if "your-" in SUBSCRIPTION_ID or "your-" in RESOURCE_GROUP or "your-" in WORKSPACE_NAME:
    print("\n⚠️  Please update the values above with your actual Azure ML workspace details!")
else:
    print("\n✅ Configuration looks good!")

## Step 3: Connect to Azure ML

This will open a browser window for you to authenticate with Azure:

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

print("🔐 Starting Azure authentication...")
print("   A browser window will open for you to sign in")

try:
    # Use interactive browser credential (most reliable)
    credential = InteractiveBrowserCredential()
    
    # Create ML client
    ml_client = MLClient(
        credential=credential,
        subscription_id=SUBSCRIPTION_ID,
        resource_group_name=RESOURCE_GROUP,
        workspace_name=WORKSPACE_NAME
    )
    
    # Test the connection
    workspace = ml_client.workspaces.get(WORKSPACE_NAME)
    
    print("\n🎉 Successfully connected to Azure ML!")
    print(f"   Workspace: {workspace.name}")
    print(f"   Location: {workspace.location}")
    print(f"   Resource Group: {workspace.resource_group}")
    
except Exception as e:
    print(f"\n❌ Connection failed: {e}")
    print("\n💡 Make sure you:")
    print("   1. Updated the workspace details in the cell above")
    print("   2. Have access to the Azure ML workspace")
    print("   3. Completed the browser authentication")

## Step 4: List Your Compute Instances

Let's see what compute resources are available in your workspace:

In [ ]:
print("🖥️  Listing compute resources...\n")

try:
    computes = list(ml_client.compute.list())
    
    if not computes:
        print("❌ No compute resources found")
        print("💡 You can create a compute instance in Azure ML Studio")
    else:
        print(f"✅ Found {len(computes)} compute resources:\n")
        
        compute_instances = []
        
        for i, compute in enumerate(computes, 1):
            compute_type = getattr(compute, 'type', 'Unknown')
            status = getattr(compute, 'provisioning_state', 'Unknown')
            
            print(f"{i}. {compute.name}")
            print(f"   Type: {compute_type}")
            print(f"   Status: {status}")
            
            if hasattr(compute, 'size'):
                print(f"   VM Size: {compute.size}")
            
            if compute_type == 'ComputeInstance':
                compute_instances.append(compute)
                
                if hasattr(compute, 'state'):
                    state = compute.state
                    print(f"   State: {state}")
                    
                    if state == 'Running':
                        print(f"   🟢 READY TO USE!")
                    elif state == 'Stopped':
                        print(f"   🔴 STOPPED (can be started)")
                    else:
                        print(f"   🟡 {state}")
            
            print()
        
        # Store compute instances for later use
        globals()['compute_instances'] = compute_instances
        
        if compute_instances:
            print(f"📊 Summary: {len(compute_instances)} compute instance(s) available")
        
except Exception as e:
    print(f"❌ Failed to list compute resources: {e}")

## Step 5: Connect to a Specific Compute Instance

Choose a compute instance to connect to:

In [ ]:
# 🔧 SPECIFY YOUR COMPUTE INSTANCE NAME
COMPUTE_NAME = "your-compute-instance-name"  # Replace with your compute instance name

print(f"🔗 Connecting to compute instance: {COMPUTE_NAME}")

try:
    compute = ml_client.compute.get(COMPUTE_NAME)
    
    print(f"\n✅ Found compute instance: {compute.name}")
    print(f"   Type: {compute.type}")
    print(f"   Status: {getattr(compute, 'provisioning_state', 'Unknown')}")
    
    if hasattr(compute, 'state'):
        state = compute.state
        print(f"   State: {state}")
        
        if state == 'Running':
            print(f"\n🎉 Compute instance is RUNNING and ready to use!")
            
            # Show available applications
            if hasattr(compute, 'applications') and compute.applications:
                print(f"\n📋 Available applications:")
                for app in compute.applications:
                    print(f"   - {app.display_name}: {app.endpoint_uri}")
            
            print(f"\n✅ You can now submit jobs to this compute instance!")
            
        elif state == 'Stopped':
            print(f"\n⚠️  Compute instance is STOPPED")
            print(f"💡 Start it from Azure ML Studio or run the next cell")
        else:
            print(f"\n⚠️  Compute instance state: {state}")
    
    # Store for later use
    globals()['selected_compute'] = compute
    
except Exception as e:
    print(f"\n❌ Failed to connect to compute '{COMPUTE_NAME}': {e}")
    print(f"\n💡 Make sure:")
    print(f"   1. The compute name is correct")
    print(f"   2. The compute exists in your workspace")
    print(f"   3. You have access permissions")

## Step 6: Start Compute Instance (if stopped)

If your compute instance is stopped, you can start it with this cell:

In [ ]:
# Only run this if your compute instance is stopped
try:
    if 'selected_compute' in globals():
        compute_name = selected_compute.name
        
        print(f"🚀 Starting compute instance: {compute_name}")
        print("   This may take a few minutes...")
        
        # Start the compute instance
        ml_client.compute.begin_start(compute_name).result()
        
        print(f"\n✅ Compute instance {compute_name} is starting!")
        print(f"💡 Check the status in Azure ML Studio")
        
    else:
        print("❌ No compute instance selected. Run the previous cell first.")
        
except Exception as e:
    print(f"❌ Failed to start compute instance: {e}")

## Step 7: Test Job Submission

Let's test if we can submit a simple job to your compute:

In [ ]:
from azure.ai.ml import command

print(f"🧪 Testing job submission capability...")

try:
    # Create a simple test job
    job = command(
        code="../notebooks",  # Use the notebooks directory
        command="python test_compute.py",
        environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
        compute=COMPUTE_NAME,
        display_name="connectivity-test"
    )
    
    print(f"✅ Job configuration is valid!")
    print(f"💡 You can submit training jobs to compute '{COMPUTE_NAME}'")
    
    # Uncomment the next line to actually submit the test job
    # submitted_job = ml_client.jobs.create_or_update(job)
    # print(f"🚀 Test job submitted: {submitted_job.name}")
    
except Exception as e:
    print(f"❌ Job configuration test failed: {e}")
    print(f"💡 This might be due to environment or compute issues")

## Step 8: Save Configuration

Save your connection details for future use:

In [ ]:
import json
import os

# Save configuration
config = {
    'subscription_id': SUBSCRIPTION_ID,
    'resource_group': RESOURCE_GROUP,
    'workspace_name': WORKSPACE_NAME,
    'compute_name': COMPUTE_NAME,
    'connection_tested': True
}

config_file = '../azure_ml_config.json'
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"💾 Configuration saved to: {os.path.abspath(config_file)}")
print(f"✅ Azure ML connection setup complete!")

print(f"\n📋 Summary:")
print(f"   Workspace: {WORKSPACE_NAME}")
print(f"   Compute: {COMPUTE_NAME}")
print(f"   Status: Connected ✅")

print(f"\n🚀 You're now ready to:")
print(f"   - Submit training jobs to Azure ML")
print(f"   - Use your compute instance for ML workloads")
print(f"   - Run the GRPO training scripts")